In [1]:
import os
import search_Qdrant as sq
import numpy as np
from dotenv import load_dotenv
from apikeys_GEMINI import APIKeyManager
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from peft import PeftModel, PeftConfig

c:\Users\hdang\.virtualenvs\machinelearning-lTTH8rYd\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
load_dotenv()
APIS_GEMINI_LIST = os.getenv('APIS_GEMINI_LIST').split(',')
key_manager = APIKeyManager(APIS_GEMINI_LIST)

In [3]:
# config = PeftConfig.from_pretrained('../model/lora/')
# tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
# base_model = AutoModelForQuestionAnswering.from_pretrained(
#     config.base_model_name_or_path,
#     trust_remote_code=True, 
#      torch_dtype=torch.float32
# )   
# # Load the Peft/Lora model
# model = PeftModel.from_pretrained(base_model, '../model/lora/')

In [4]:
tokenizer=AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")
model =AutoModelForQuestionAnswering.from_pretrained('quanghuy123/fine-tuning-bert-for-Question-Answer')

In [5]:
MODEL_RERANK = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
rerank_model = SentenceTransformer(MODEL_RERANK)
MAX_LENGTH = 512
STRIDE = 400
N_BEST = 200
MAX_ANSWER_LENGTH = 2000

In [6]:
def predict(contexts, question):
    answer_final = []
    for context in contexts:
        inputs = tokenizer(
            question,
            context,
            max_length=MAX_LENGTH,
            truncation="only_second",
            stride=STRIDE,
            return_offsets_mapping=True,
            padding="max_length",
            return_tensors="pt"
        )
        with torch.no_grad():
            outputs = model(**{k: v for k, v in inputs.items() if k in ['input_ids', 'attention_mask']})
        start_logits = outputs.start_logits.squeeze().cpu().numpy()
        end_logits = outputs.end_logits.squeeze().cpu().numpy()
        offsets = inputs["offset_mapping"][0].cpu().numpy()
        answers = []
        start_indexes = np.argsort(start_logits)[-N_BEST:][::-1].tolist()
        end_indexes = np.argsort(end_logits)[-N_BEST:][::-1].tolist()

        for start_index in start_indexes:
            for end_index in end_indexes:
                if end_index < start_index or end_index - start_index + 1 > MAX_ANSWER_LENGTH:
                    continue
                if offsets[start_index][0] is not None and offsets[end_index][1] is not None:
                    answer_text = context[offsets[start_index][0]: offsets[end_index][1]].strip()
                    if answer_text:
                        answer = {
                            "text": answer_text,
                            "score": start_logits[start_index] + end_logits[end_index],
                        }
                        answers.append(answer)
        if answers:
            answers.sort(key=lambda x: x["score"], reverse=True)
            best_answer = answers[0]['text']
            if best_answer not in answer_final:
                answer_final.append(best_answer)
        else: 
            return "Không có câu trả lời"
    return answer_final  
 

In [7]:
question="Hãy cho tôi biết đối tượng tham gia bảo hiểm xã hội bắt buộc và bảo hiểm xã hội tự nguyện là ai ?"
contexts=sq.search_Article(question,key_manager)
for context in contexts:
    print(context,"\n----------------------------------------------------------------")


Các kết quả trả về:

Điều 27. Hồ sơ đăng ký tham gia bảo hiểm xã hội bắt buộc và bảo hiểm xã hội tự nguyện
1. Hồ sơ đăng ký tham gia bảo hiểm xã hội bắt buộc của đối tượng tham gia bảo hiểm xã hội bắt buộc, trừ đối tượng quy định tại điểm m và điểm n khoản 1 Điều 2 của Luật này bao gồm:
a) Tờ khai đăng ký tham gia bảo hiểm xã hội của người sử dụng lao động kèm theo danh sách người lao động tham gia bảo hiểm xã hội;
b) Tờ khai đăng ký tham gia bảo hiểm xã hội của người lao động.
2. Hồ sơ đăng ký tham gia bảo hiểm xã hội bắt buộc của đối tượng quy định tại điểm m và điểm n khoản 1 Điều 2 của Luật này nếu nộp qua người sử dụng lao động thì thực hiện theo quy định tại khoản 1 Điều này, nếu tự nộp thì hồ sơ là tờ khai quy định tại điểm b khoản 1 Điều này.
3. Hồ sơ đăng ký tham gia bảo hiểm xã hội tự nguyện là tờ khai đăng ký tham gia bảo hiểm xã hội của người tham gia bảo hiểm xã hội tự nguyện. 
----------------------------------------------------------------
Điều 3. Giải thích từ ngữ
Tron

In [8]:
results=predict(contexts,question)
for result in results:
    print(result,"\n----------------------------------------------------------------")

Hồ sơ đăng ký tham gia bảo hiểm xã hội tự nguyện là tờ khai đăng ký tham gia bảo hiểm xã hội của người tham gia bảo hiểm xã hội tự nguyện. 
----------------------------------------------------------------
Bảo hiểm xã hội là sự bảo đảm thay thế hoặc bù đắp một phần thu nhập của người tham gia bảo hiểm xã hội khi họ bị giảm hoặc mất thu nhập do ốm đau, thai sản, tai nạn lao động, bệnh nghề nghiệp, khi nghỉ hưu hoặc chết, trên cơ sở đóng vào quỹ bảo hiểm xã hội hoặc do ngân sách nhà nước bảo đảm.
2. Trợ cấp hưu trí xã hội là loại hình bảo hiểm xã hội do ngân sách nhà nước bảo đảm cho người cao tuổi đủ điều kiện theo quy định của Luật này.
3. Bảo hiểm xã hội bắt buộc là loại hình bảo hiểm xã hội do Nhà nước tổ chức mà người lao động, người sử dụng lao động thuộc đối tượng tham gia bảo hiểm xã hội bắt buộc phải tham gia.
4. Bảo hiểm xã hội tự nguyện là loại hình bảo hiểm xã hội do Nhà nước tổ chức mà công dân Việt Nam tự nguyện tham gia và được lựa chọn mức đóng, phương thức đóng phù hợp vớ

In [9]:
def rerank_By_Cosin_TF_IDF(user_query, documents):
    user_query_embedding = rerank_model.encode(user_query)
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)  
    user_query_tfidf = vectorizer.transform([user_query]) 
    tfidf_scores = cosine_similarity(user_query_tfidf, tfidf_matrix).flatten() 
    
    combined_scores = []
    for i, document in enumerate(documents):
        document_embedding = rerank_model.encode(document) 
        cos_sim = cosine_similarity([user_query_embedding], [document_embedding])[0][0]  
        combined_score = 0.7 * cos_sim + 0.3 * tfidf_scores[i]  
        combined_scores.append(combined_score)
    
    results_with_scores = [(documents[i], combined_scores[i]) for i in range(len(documents))]
    results_with_scores.sort(key=lambda x: x[1], reverse=True)
    
    top_results = [results_with_scores[i][0] for i in range(min(2, len(documents)))]
    return top_results

In [10]:
finals=rerank_By_Cosin_TF_IDF(question,results)
for final in finals:
    print(final,"\n----------------------------------------------------------------")

Hồ sơ đăng ký tham gia bảo hiểm xã hội tự nguyện là tờ khai đăng ký tham gia bảo hiểm xã hội của người tham gia bảo hiểm xã hội tự nguyện. 
----------------------------------------------------------------
Bảo hiểm xã hội là sự bảo đảm thay thế hoặc bù đắp một phần thu nhập của người tham gia bảo hiểm xã hội khi họ bị giảm hoặc mất thu nhập do ốm đau, thai sản, tai nạn lao động, bệnh nghề nghiệp, khi nghỉ hưu hoặc chết, trên cơ sở đóng vào quỹ bảo hiểm xã hội hoặc do ngân sách nhà nước bảo đảm.
2. Trợ cấp hưu trí xã hội là loại hình bảo hiểm xã hội do ngân sách nhà nước bảo đảm cho người cao tuổi đủ điều kiện theo quy định của Luật này.
3. Bảo hiểm xã hội bắt buộc là loại hình bảo hiểm xã hội do Nhà nước tổ chức mà người lao động, người sử dụng lao động thuộc đối tượng tham gia bảo hiểm xã hội bắt buộc phải tham gia.
4. Bảo hiểm xã hội tự nguyện là loại hình bảo hiểm xã hội do Nhà nước tổ chức mà công dân Việt Nam tự nguyện tham gia và được lựa chọn mức đóng, phương thức đóng phù hợp vớ